In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
# for train dataset preprocessing

train = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/train_set.csv')
train["date_time"]=pd.to_datetime(train.date_time_IST)

train_data = pd.DataFrame()
train_data['date_time'] = train['date_time_IST']
train_data['temp']= train['tmpc']
train_data['dew'] = train['dwpc']
train_data['humidity'] = train['relh']
train_data['pressure'] = train['pressure']
train_data['visibility'] = train['vsbykm']
#coverting type of date_time column
train_data["date_time"]=pd.to_datetime(train_data.date_time)
# adding fog duration var
train_data["endtime"] = train_data["date_time"].shift(1)
train_data.loc[train_data.visibility<2.0,'fog_duration']=(train_data['date_time']-train_data['endtime']).astype('timedelta64[s]')#astype here to convert time to float sec
train_data.loc[train_data.visibility>=2.0,'fog_duration']=0
# adding energy_loss var
train_data.loc[train_data.visibility<2.0, 'energy_loss']=train_data['fog_duration']*(1-np.exp(-0.05*3/train_data['visibility']))
train_data.loc[train_data.visibility>=2.0,'energy_loss']=0
# fog index
train_data["fog_index"]= train_data["energy_loss"]/train_data["fog_duration"]
train_data.loc[train_data.fog_duration==0,'fog_index']=0
train_data.dropna(inplace = True)

# resampled in 6h
#making datetime as index
train_data.set_index("date_time",inplace=True)
train_data_6h=pd.DataFrame()
train_data_6h["avg_air_temp"]=train_data["temp"].resample("6H",loffset="4h").mean()
# tried for loffset from 0 to 6 but the number of na vaues are constant is 16035 out of 23616 about 67% data reduced , 15666 data is from non fog month , out of 7935 winter fog data 369 is null which is 4.6%
train_data_6h["avg_dew_point"]=train_data["dew"].resample("6h",loffset="4h").mean()

train_data_6h["avg_relative_humidity"] = train_data["humidity"].resample("6h",loffset="4h").mean()

train_data_6h["avg_pressure"] = train_data["pressure"].resample("6h",loffset="4h").mean()

train_data_6h["avg_visibility"] = train_data["visibility"].resample("6h",loffset="4h").mean()

train_data_6h["energy_loss"]=train_data.resample('6h',loffset="4h").agg({'energy_loss':'sum'}) #above and below will work as same

train_data_6h["fog_duration"]=train_data.resample('6h',loffset="4h").agg({'fog_duration':'sum'})

#Adding Fog_month variable
train_data_6h["fog_month"]=0
train_data_6h.loc[(train_data_6h.index.month==12)|(train_data_6h.index.month==1),"fog_month"]=1
train_data_6h.dropna(inplace = True)

# fog_index variable for 6 hours
train_data_6h['fog_index']=train_data_6h['energy_loss']/train_data_6h['fog_duration']
train_data_6h.loc[train_data_6h.fog_duration==0,'fog_index']=0
train_data_6h.dropna(inplace = True)
# show that there is not continous data of interval 30 min there are many ranges which are missing
# solution is to drop the data for the Na values

In [ ]:
train_data_6h.head()

,avg_air_temp,avg_dew_point,avg_relative_humidity,avg_pressure,avg_visibility,energy_loss,fog_duration,fog_month,fog_index
date_time,,,,,,,,,
2000-01-01 04:00:00,10.000000,10.000000,100.000000,30.011667,0.193121,10693.704233,19800.0,1,0.540086
2000-01-01 10:00:00,10.860000,10.430000,97.136333,30.076000,0.410382,8798.058547,21600.0,1,0.407318
2000-01-01 16:00:00,13.175000,11.625000,90.555000,30.011250,1.425607,2349.704586,21600.0,1,0.108783
2000-01-01 22:00:00,9.250000,9.125000,98.656250,30.038750,0.193121,11665.859163,21600.0,1,0.540086
2000-01-02 04:00:00,9.636364,8.727273,94.547727,30.031364,0.193121,11665.859163,21600.0,1,0.540086


In [ ]:
train_data_6h.to_csv('/content/drive/MyDrive/Dataset/Current dataset/train_preprocessed.csv')

In [ ]:
# for test dataset preprocessing

train = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/test_set.csv')
train["date_time"]=pd.to_datetime(train.date_time_IST)

train_data = pd.DataFrame()
train_data['date_time'] = train['date_time_IST']
train_data['temp']= train['tmpc']
train_data['dew'] = train['dwpc']
train_data['humidity'] = train['relh']
train_data['pressure'] = train['pressure']
train_data['visibility'] = train['vsbykm']
#coverting type of date_time column
train_data["date_time"]=pd.to_datetime(train_data.date_time)
# adding fog duration var
train_data["endtime"] = train_data["date_time"].shift(1)
train_data.loc[train_data.visibility<2.0,'fog_duration']=(train_data['date_time']-train_data['endtime']).astype('timedelta64[s]')#astype here to convert time to float sec
train_data.loc[train_data.visibility>=2.0,'fog_duration']=0
# adding energy_loss var
train_data.loc[train_data.visibility<2.0, 'energy_loss']=train_data['fog_duration']*(1-np.exp(-0.05*3/train_data['visibility']))
train_data.loc[train_data.visibility>=2.0,'energy_loss']=0
# fog index
train_data["fog_index"]= train_data["energy_loss"]/train_data["fog_duration"]
train_data.loc[train_data.fog_duration==0,'fog_index']=0
train_data.dropna(inplace = True)

# resampled in 6h
#making datetime as index
train_data.set_index("date_time",inplace=True)
train_data_6h=pd.DataFrame()
train_data_6h["avg_air_temp"]=train_data["temp"].resample("6H",loffset="4h").mean()

train_data_6h["avg_dew_point"]=train_data["dew"].resample("6h",loffset="4h").mean()

train_data_6h["avg_relative_humidity"] = train_data["humidity"].resample("6h",loffset="4h").mean()

train_data_6h["avg_pressure"] = train_data["pressure"].resample("6h",loffset="4h").mean()

train_data_6h["avg_visibility"] = train_data["visibility"].resample("6h",loffset="4h").mean()

train_data_6h["energy_loss"]=train_data.resample('6h',loffset="4h").agg({'energy_loss':'sum'}) #above and below will work as same

train_data_6h["fog_duration"]=train_data.resample('6h',loffset="4h").agg({'fog_duration':'sum'})

#Adding Fog_month variable
train_data_6h["fog_month"]=0
train_data_6h.loc[(train_data_6h.index.month==12)|(train_data_6h.index.month==1),"fog_month"]=1
train_data_6h.dropna(inplace = True)

# fog_index variable for 6 hours
train_data_6h['fog_index']=train_data_6h['energy_loss']/train_data_6h['fog_duration']
train_data_6h.loc[train_data_6h.fog_duration==0,'fog_index']=0
train_data_6h.dropna(inplace = True)
# show that there is not continous data of interval 30 min there are many ranges which are missing
# solution is to drop the data for the Na values

<ipython-input-12-01a0106d7c79>:31: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to_offset
>>> df = df.resample(freq="3s").mean()
>>> df.index = df.index.to_timestamp() + to_offset("8H")

  train_data_6h["avg_air_temp"]=train_data["temp"].resample("6H",loffset="4h").mean()
<ipython-input-12-01a0106d7c79>:33: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to_offset
>>> df = df.resample(freq="3s").mean()
>>> df.index = df.index.to_timestamp() + to_offset("8H")

  train_data_6h["avg_dew_point"]=train_data["dew"].resample("6h",loffset="4h").mean()
<ipython-input-12-01a0106d7c79>:35: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to

In [ ]:
train_data_6h.sample(4)

,avg_air_temp,avg_dew_point,avg_relative_humidity,avg_pressure,avg_visibility,energy_loss,fog_duration,fog_month,fog_index
date_time,,,,,,,,,
2022-11-19 04:00:00,13.833333,12.666667,92.696667,30.00000,1.995582,1564.068075,21600.0,0,0.072411
2022-12-05 10:00:00,15.750000,14.208333,90.885833,30.05875,0.548517,7135.012457,21600.0,1,0.330325
2021-01-08 04:00:00,15.583333,14.500000,93.282500,29.89250,0.855632,5997.433886,21600.0,1,0.277659
2019-12-13 04:00:00,17.166667,16.083333,93.391667,29.90500,1.291495,2649.558433,21600.0,1,0.122665


In [ ]:
train_data_6h.to_csv('/content/drive/MyDrive/Dataset/Current dataset/test_preprocessed.csv')

In [ ]:
# for val dataset preprocessing

train = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/val_set.csv')
train["date_time"]=pd.to_datetime(train.date_time_IST)

train_data = pd.DataFrame()
train_data['date_time'] = train['date_time_IST']
train_data['temp']= train['tmpc']
train_data['dew'] = train['dwpc']
train_data['humidity'] = train['relh']
train_data['pressure'] = train['pressure']
train_data['visibility'] = train['vsbykm']
#coverting type of date_time column
train_data["date_time"]=pd.to_datetime(train_data.date_time)
# adding fog duration var
train_data["endtime"] = train_data["date_time"].shift(1)
train_data.loc[train_data.visibility<2.0,'fog_duration']=(train_data['date_time']-train_data['endtime']).astype('timedelta64[s]')#astype here to convert time to float sec
train_data.loc[train_data.visibility>=2.0,'fog_duration']=0
# adding energy_loss var
train_data.loc[train_data.visibility<2.0, 'energy_loss']=train_data['fog_duration']*(1-np.exp(-0.05*3/train_data['visibility']))
train_data.loc[train_data.visibility>=2.0,'energy_loss']=0
# fog index
train_data["fog_index"]= train_data["energy_loss"]/train_data["fog_duration"]
train_data.loc[train_data.fog_duration==0,'fog_index']=0
train_data.dropna(inplace = True)

# resampled in 6h
#making datetime as index
train_data.set_index("date_time",inplace=True)
train_data_6h=pd.DataFrame()
train_data_6h["avg_air_temp"]=train_data["temp"].resample("6H",loffset="4h").mean()

train_data_6h["avg_dew_point"]=train_data["dew"].resample("6h",loffset="4h").mean()

train_data_6h["avg_relative_humidity"] = train_data["humidity"].resample("6h",loffset="4h").mean()

train_data_6h["avg_pressure"] = train_data["pressure"].resample("6h",loffset="4h").mean()

train_data_6h["avg_visibility"] = train_data["visibility"].resample("6h",loffset="4h").mean()

train_data_6h["energy_loss"]=train_data.resample('6h',loffset="4h").agg({'energy_loss':'sum'}) #above and below will work as same

train_data_6h["fog_duration"]=train_data.resample('6h',loffset="4h").agg({'fog_duration':'sum'})

#Adding Fog_month variable
train_data_6h["fog_month"]=0
train_data_6h.loc[(train_data_6h.index.month==12)|(train_data_6h.index.month==1),"fog_month"]=1
train_data_6h.dropna(inplace = True)

# fog_index variable for 6 hours
train_data_6h['fog_index']=train_data_6h['energy_loss']/train_data_6h['fog_duration']
train_data_6h.loc[train_data_6h.fog_duration==0,'fog_index']=0
train_data_6h.dropna(inplace = True)

# show that there is not continous data of interval 30 min there are many ranges which are missing
# solution is to drop the data for the Na values

<ipython-input-17-fecaafab9526>:31: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to_offset
>>> df = df.resample(freq="3s").mean()
>>> df.index = df.index.to_timestamp() + to_offset("8H")

  train_data_6h["avg_air_temp"]=train_data["temp"].resample("6H",loffset="4h").mean()
<ipython-input-17-fecaafab9526>:33: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to_offset
>>> df = df.resample(freq="3s").mean()
>>> df.index = df.index.to_timestamp() + to_offset("8H")

  train_data_6h["avg_dew_point"]=train_data["dew"].resample("6h",loffset="4h").mean()
<ipython-input-17-fecaafab9526>:35: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to

In [ ]:
train_data_6h.to_csv('/content/drive/MyDrive/Dataset/Current dataset/val_preprocessed.csv')

In [ ]:
train_data_6h.sample(4)

,avg_air_temp,avg_dew_point,avg_relative_humidity,avg_pressure,avg_visibility,energy_loss,fog_duration,fog_month,fog_index
date_time,,,,,,,,,
2016-12-03 04:00:00,16.909091,16.727273,98.878182,30.030000,0.052669,20333.389581,21600.0,1,0.941361
2018-12-30 04:00:00,6.601852,4.444444,86.254259,30.096944,1.670137,2697.431482,32400.0,1,0.083254
2016-11-10 22:00:00,19.166667,14.875000,76.483333,29.896250,0.941464,3213.731667,21600.0,0,0.148784
2019-02-27 22:00:00,15.041667,12.750000,86.706667,29.885833,2.286604,651.695031,9000.0,0,0.072411


In [ ]:
train_data_6h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1430 entries, 2016-11-01 04:00:00 to 2019-02-28 22:00:00
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   avg_air_temp           1430 non-null   float64
 1   avg_dew_point          1430 non-null   float64
 2   avg_relative_humidity  1430 non-null   float64
 3   avg_pressure           1430 non-null   float64
 4   avg_visibility         1430 non-null   float64
 5   energy_loss            1430 non-null   float64
 6   fog_duration           1430 non-null   float64
 7   fog_month              1430 non-null   int64  
 8   fog_index              1430 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 111.7 KB
